# Lumma Stealer Obfuscation
> Taking a look at obfuscation in the latest version of lumma

- toc: true 
- badges: true
- categories: [lumma,obfuscation,cff,ida]

## Overview

[Lumma](https://telegra.ph/LummaC2---universal-stealer-a-malware-for-professionals-07-27) has been around for a [while](https://www.esentire.com/blog/the-case-of-lummac2-v4-0) initially starting out with plaintext strings and simple to reverse code, but as they continue to evolve they added some [control flow flattening](https://outpost24.com/blog/lummac2-anti-sandbox-technique-trigonometry-human-detection/#h-control-flow-flattening) (CFF) and string encryption. With the latest build more obfuscation has been added which introduces opaque predicates to the CFF. We are going to attempt to use IDA scripting to remove this obfuscation.

## Sample
`18a065b740da441c636bce23fd72fc0f68e935956131973f15bf4918e317bf03` [[UnpacMe](https://www.unpac.me/results/41d8f31a-3b36-4b77-8b7c-a2d7a81a7034#/)]


## Analysis

The opaque predicates are setup using a jump table with encrypted addresses, for direct jumps the address is simply moved into a register, decrypted, then jumped to. For conditional jumps the jump table is used like an index to locate the correct address, which is then decrypted and jumped to.

```
00433500      mov     eax, dword_4426C0
00433505      mov     ecx, 439EAE97h
0043350A      xor     ecx, dword_4426C8
00433510      add     eax, ecx
00433512      inc     eax
00433513      jmp     eax

0043351C      mov     eax, dword_442714
00433521      mov     ecx, 0DEA4552Dh
00433526      xor     ecx, dword_442718
0043352C      add     eax, ecx
0043352E      inc     eax
0043352F      nop
00433530      jmp     eax

00433399      movzx   eax, al
0043339C      mov     eax, dword_4426D4[eax*4]
004333A3      mov     ecx, 42CEBA5h
004333A8      xor     ecx, dword_4426DC
004333AE      add     eax, ecx
004333B0      inc     eax
004333B1      jmp     eax

00433312      test    eax, eax
00433314      setz    cl
00433317      mov     ecx, dword_4426A0[ecx*4]
0043331E      mov     edx, 0AD7AC4AAh
00433323      xor     edx, dword_4426A8
00433329      add     ecx, edx
0043332B      inc     ecx
0043332C      jmp     ecx
```

### Algorithm
- Linear disassembly 
- Track memory moves to regs from jump table memory locations `.data`
- Reset memory tracking for control flow operations JMP,CALL,RET
- When we have a JMP to a register use tracked memory location as emulation start and emulate jump address
- For conditional jumps emulated both values for the index reg for the jump table (0,1) 
- Patch jumps, for condition test the index register and patch conditional jump

**TO BE CONTINUED....**
